<a href="https://colab.research.google.com/github/akibkhan1/skin-lesion-classification/blob/main/DU_Net_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Necessary Imports

In [ ]:
import os
import cv2
import zipfile
from glob import glob
import random
from tqdm import tqdm
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from shutil import copyfile, move
from IPython.display import clear_output

## Cloning Author Github Repo

In [ ]:
!git clone 'https://github.com/DebeshJha/2020-CBMS-DoubleU-Net'

Cloning into '2020-CBMS-DoubleU-Net'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 93 (delta 42), reused 2 (delta 0), pack-reused 0
Unpacking objects: 100% (93/93), done.


## Our Weights

In [ ]:
!gdown --id 1-EITJRQFgWQBRTa1VgcEtxsikftwkCoR

Downloading...
From: https://drive.google.com/uc?id=1-EITJRQFgWQBRTa1VgcEtxsikftwkCoR
To: /content/2020-CBMS-DoubleU-Net/model.05-0.42.h5
353MB [00:06, 54.8MB/s]


In [ ]:
%cd /content/2020-CBMS-DoubleU-Net

/content/2020-CBMS-DoubleU-Net


## Downloading some necesaary scripts(before running the next cell delete the existing train.py, predict.py, utils.py files)

After downloading and unzipping we will get modified 
train.py, predict.py, utils.py

In [ ]:
%cd /content/2020-CBMS-DoubleU-Net
!gdown --id 13fk3N0ZQ_4cujmcVeP8VY8xrBywrP-iH
!unzip necesaary_scripts.zip

## Downloading Data

In [ ]:
!gdown --id 19BErvkVNU02jPVlbPtjlfNkm_dg7Y7-t

Downloading...
From: https://drive.google.com/uc?id=19BErvkVNU02jPVlbPtjlfNkm_dg7Y7-t
To: /content/2020-CBMS-DoubleU-Net/noaugmentation_512_384_dunet.zip
126MB [00:02, 43.7MB/s]


In [ ]:
!unzip /content/2020-CBMS-DoubleU-Net/noaugmentation_512_384_dunet.zip

## Downloading Weights provided by Author

In [ ]:
!gdown --id 1jjMYoMkbb866-1qh3bD546TVp-ZjKs9w

Downloading...
From: https://drive.google.com/uc?id=1jjMYoMkbb866-1qh3bD546TVp-ZjKs9w
To: /content/2020-CBMS-DoubleU-Net/DoubleUNet.zip
322MB [00:08, 38.2MB/s]


In [ ]:
!unzip DoubleUNet.zip

Archive:  DoubleUNet.zip
   creating: DoubleUNet/
   creating: DoubleUNet/files/
  inflating: DoubleUNet/files/model.h5  
   creating: DoubleUNet/files/.ipynb_checkpoints/
  inflating: DoubleUNet/files/.ipynb_checkpoints/data-checkpoint.csv  


## Run Prediction

In [ ]:
!python predict.py

2021-07-27 18:25:22.034572: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-27 18:25:23.592145: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-27 18:25:23.598278: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-27 18:25:23.598910: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-07-27 18:25:23.598958: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-27 18:25:23.601506: I tensorflow/stream_executor/platform/default

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.metrics import *
from glob import glob
from sklearn.model_selection import train_test_split
from model import build_model
from utils import *
from metrics import *

In [ ]:
metrics = [
    dice_coef,
    iou,
    Recall(),
    Precision()
]

In [ ]:
new_model = tf.keras.models.load_model('/content/2020-CBMS-DoubleU-Net/model.05-0.42.h5', custom_objects={'dice_loss':dice_loss,'iou':iou,'dice_coef':dice_coef})